In [ ]:
import torch

# device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
device = "cpu"
print(f"Using device: {device}")

# Set the default tensor type and device for all computations
torch.set_default_device(device)

In [ ]:
from the_well.data import WellDataset

the_well = WellDataset(
    well_base_path="../data/the_well/datasets",
    well_dataset_name="active_matter",
    well_split_name="test",
    n_steps_input=5,
    n_steps_output=1,
    
)


In [ ]:
batch = next(iter(the_well))
x = batch["input_fields"]
x.shape

In [ ]:
x = torch.arange(0, 80*1*1*11).reshape(80, 1, 1, 11)
x.shape


In [ ]:
import torch
from torch.utils.data import Dataset
import h5py

from autoemulate.core.types import TensorLike

import torch
from torch.utils.data import Dataset
import h5py

from autoemulate.core.types import TensorLike

class AutoEmulateDataset(Dataset):
    def __init__(
        self,
        data_path: str,
        n_steps_input: int,
        n_steps_output: int,
        stride: int = 1,
        input_channel_idxs: tuple[int, ...] | None = None,
        output_channel_idxs: tuple[int, ...] | None = None,
    ):
        """
        Args:
            data_path: Path to HDF5 file
            t_in: Number of input timesteps
            t_out: Number of output timesteps  
            stride: Stride between sequences
        """
        self.n_steps_input = n_steps_input
        self.n_steps_output = n_steps_output
        self.stride = stride
        self.input_channel_idxs = input_channel_idxs
        self.output_channel_idxs = output_channel_idxs
        
        # Load data
        with h5py.File(data_path, 'r') as f:
            self.data: TensorLike = torch.Tensor(f['data'][:])  # [N, T, W, H, C]
            print(f"Loaded data shape: {self.data.shape}")
            # TODO: add the constant scalars
            self.constant_scalars: TensorLike = torch.Tensor(f['constant_scalars'][:]) if 'constant_scalars' in f else None # [N, C]
            # TODO: add the constant fields
            # self.constant_fields = torch.Tensor(f['data'][:])  # [N, W, H, C]
        
        # Destructured here
        self.n_trajectories, self.n_timesteps, self.width, self.height, self.n_channels = self.data.shape
        
        # Pre-compute all subtrajectories for efficient indexing
        self.all_input_fields = []
        self.all_output_fields = []
        self.all_constant_scalars = []
        
        for traj_idx in range(self.n_trajectories):
            # Create subtrajectories for this trajectory
            fields = (
                self.data[traj_idx]
                .unfold(0, self.n_steps_input + self.n_steps_output, self.stride)
                .permute(0, -1, 1, 2, 3) # [num_subtrajectories, T_in + T_out, W, H, C]
            )
            
            # Split into input and output
            input_fields = fields[:, :self.n_steps_input, ...]  # [num_subtrajectories, T_in, W, H, C]
            output_fields = fields[:, self.n_steps_input:, ...]  # [num_subtrajectories, T_out, W, H, C]
            
            # Store each subtrajectory separately
            for sub_idx in range(input_fields.shape[0]):
                self.all_input_fields.append(input_fields[sub_idx])  # [T_in, W, H, C]
                self.all_output_fields.append(output_fields[sub_idx])  # [T_out, W, H, C]
                
                # Handle constant scalars
                if self.constant_scalars is not None:
                    self.all_constant_scalars.append(self.constant_scalars[traj_idx])
                else:
                    self.all_constant_scalars.append(torch.tensor([]))
        
        print(f"Created {len(self.all_input_fields)} subtrajectory samples")
        print(f"Each input sample shape: {self.all_input_fields[0].shape}")
        print(f"Each output sample shape: {self.all_output_fields[0].shape}")
    
    def __len__(self):
        return len(self.all_input_fields)
    
    def __getitem__(self, idx):
        return {
            "input_fields": self.all_input_fields[idx],
            "output_fields": self.all_output_fields[idx],
            # "constant_scalars": self.all_constant_scalars[idx],
            # TODO: add this
            # "constant_fields": self.all_constant_fields[idx],
        }
    

class MHDDataset(AutoEmulateDataset):
    """PyTorch Dataset for MHD data"""

    def __init__(self, data_path: str, t_in: int = 5, t_out: int = 10, stride: int = 1):
        super().__init__(data_path, n_steps_input=t_in, n_steps_output=t_out, stride=stride)


class MyFNO():
    def forward(self, batch):

        # Implement the forward pass
        x = batch["input_fields"]
        # constant_scalars = batch["constant_scalars"]
        constant_scalars = batch["constant_params"]
        return x

In [ ]:

from torch.utils.data import DataLoader

train_loader = DataLoader(the_well, batch_size=1)

batch = next(iter(train_loader))


In [ ]:
batch.keys()

In [ ]:
batch["input_fields"].shape

In [ ]:
batch["output_time_grid"]


In [ ]:

batch["input_fields"].shape, batch["output_fields"].shape


In [ ]:

batch["input_fields"][0, 0, :, :, 0].shape


In [ ]:

import matplotlib.pyplot as plt

plt.imshow(batch["input_fields"][0, 0, :, :, 0].cpu())


In [ ]:
# Example with fusion

dataset = MHDDataset("../data/fusion/mhd_dataset.h5")
batch_fusion = next(iter(DataLoader(dataset)))



In [ ]:
batch_fusion["input_fields"].shape, batch_fusion["output_fields"].shape

In [ ]:

from neuralop.models import FNO

model = FNO(
    n_modes=(2, 16, 16),
    hidden_channels=16,
    in_channels=4,
    out_channels=1,
)


In [ ]:
from autoemulate.core.types import TensorLike
from autoemulate.emulators.base import PyTorchBackend

import torch

def prepare_batch(sample, channels = (0,), with_constants=True, with_time=False):
    # Get input fields, constant scalars and output fields
    x = sample["input_fields"][:, :, :, :, channels]  # [batch, time, height, width, len(channels)]
    constant_scalars = sample["constant_scalars"]  # [batch, n_constants]
    y = sample["output_fields"][:, :, :, :, channels]  # [batch, time, height, width, len(channels)]
    
    # Permute both x and y
    x = x.permute(0, 4, 1, 2, 3)  # [batch, len(channels), time, height, width]
    y = y.permute(0, 4, 1, 2, 3)  # [batch, len(channels), time, height, width]

    # Only add constants to input, not output
    if with_constants:
        # Assign spatio-temporal dims to constants
        time_window, height, width = x.shape[2], x.shape[3], x.shape[4]
        n_constants = constant_scalars.shape[-1]

        # Add spatio-temporal dims to constants
        c_broadcast = constant_scalars.reshape(1, n_constants, 1, 1, 1).expand(1, n_constants, time_window, height, width)
        
        # Concatenate along channel dimension
        x = torch.cat([x, c_broadcast], dim=1)

    if not with_time:
        # Take last time step for both input and output
        return x[:, :, -1, :, :], y[:, :, -1, :, :]
    # Otherwise include time
    return x, y

class FNOEmulator(PyTorchBackend):
    def __init__(self, x, y, *args, **kwargs):
        self.model = FNO(**kwargs)

    def _fit(self, x: DataLoader, y: DataLoader):
        channels = (0,)  # Which channel to use
        print_shapes = False
        for idx, batch in enumerate(train_loader):
            # Prepare input with constants
            x, y = prepare_batch(
                batch, channels=channels, with_constants=True, with_time=True
            )
            
            # Predictions
            y_pred = model(x)

            # Print shapes
            if print_shapes:
                print(x.shape, y.shape, y_pred.shape)
            
            # Get loss
            # Take the first time idx as the next time step prediction
            loss = self.loss_fn(y_pred[:, :, :1, ...], y)

            loss.backward()
            self.optimizer.step()
            self.optimizer.zero_grad()

            print(f"sample {idx:5d}, loss: {loss.item():.5e}")

    def forward(self, x: TensorLike):
        return self.model(x)
    
    def _predict(self, x, with_grad):
        return super()._predict(x, with_grad)


In [ ]:
x = batch["input_fields"][:, :, :, :, :1]  # [batch, time, height, width, channels]
y = batch["output_fields"][:, :, :, :, :1]  # [batch, time, height, width, channels]
x.shape, y.shape


In [ ]:
x = x.permute(0, 4, 1, 2, 3)  # Convert to [batch, channels, time, height, width]
x.shape


In [ ]:
import torch

def prepare_batch(sample, channels = (0,), with_constants=True, with_time=False):
    # Get input fields, constant scalars and output fields
    x = sample["input_fields"][:, :, :, :, channels]  # [batch, time, height, width, len(channels)]
    constant_scalars = sample["constant_scalars"]  # [batch, n_constants]
    y = sample["output_fields"][:, :, :, :, channels]  # [batch, time, height, width, len(channels)]
    
    # Permute both x and y
    x = x.permute(0, 4, 1, 2, 3)  # [batch, len(channels), time, height, width]
    y = y.permute(0, 4, 1, 2, 3)  # [batch, len(channels), time, height, width]

    # Only add constants to input, not output
    if with_constants:
        # Assign spatio-temporal dims to constants
        time_window, height, width = x.shape[2], x.shape[3], x.shape[4]
        n_constants = constant_scalars.shape[-1]

        # Add spatio-temporal dims to constants
        c_broadcast = constant_scalars.reshape(1, n_constants, 1, 1, 1).expand(1, n_constants, time_window, height, width)
        
        # Concatenate along channel dimension
        x = torch.cat([x, c_broadcast], dim=1)

    if not with_time:
        # Take last time step for both input and output
        return x[:, :, -1, :, :], y[:, :, -1, :, :]
    # Otherwise include time
    return x, y


In [ ]:
# Without time
x_with_constants, y = prepare_batch(batch, channels=(0,), with_time=True)
print(f"Concatenated x shape: {x_with_constants.shape}")
print(f"Output y shape: {y.shape}")


In [ ]:
# With time
x_with_constants, y = prepare_batch(batch, channels=(0,), with_time=True)
print(f"Concatenated x shape: {x_with_constants.shape}")
print(f"Output y shape: {y.shape}")


In [ ]:
prepare_batch(batch, with_time=True)[0].shape

In [ ]:

# Pass through model
model(prepare_batch(batch, with_time=True)[0]).shape


In [ ]:
from torch.optim import AdamW
from torch.nn import MSELoss

# Create new model
model = FNO(
    n_modes=(2, 16, 16),
    hidden_channels=16,
    in_channels=4,
    out_channels=1,
).to(device)

# Explicitly set shuffle=False to ensure monotonic ordering
train_loader = DataLoader(the_well, batch_size=1, shuffle=False)

optimizer = AdamW(
    model.parameters(),
    lr=1e-2,
    # weight_decay=1e-4
)

loss_fn = MSELoss().to(device)
channels = (0,)  # Which channel to use
print_shapes = False
for idx, batch in enumerate(train_loader):
    # Prepare input with constants
    x, y = prepare_batch(batch, channels=channels, with_constants=True, with_time=True)
    
    # Predictions
    y_pred = model(x)

    # Print shapes
    if print_shapes:
        print(x.shape, y.shape, y_pred.shape)
    
    # Get loss
    # Take the first time idx as the next time step prediction
    loss = loss_fn(y_pred[:, :, :1, ...], y)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    print(f"sample {idx:5d}, loss: {loss.item():.5e}")

    # Break after a few samples for testing
    if idx >= 100:
        break